# Veri Madenciliğinde İleri Konular - Proje

**Veri Seti:** İlaçlar ve Yan Etkileri (real_drug_dataset.csv)

Bu projede, ilaçlar ve yan etkileri içeren veri seti üzerinde kapsamlı bir veri madenciliği analizi gerçekleştireceğiz.

## Gerekli Kütüphanelerin İçe Aktarılması

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Görselleştirme ayarları
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Pandas görüntüleme ayarları
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("Kütüphaneler başarıyla yüklendi!")

## Google Colab - Otomatik Veri Seti Yükleme

**Google Colab kullanıyorsanız:** Veri seti otomatik olarak GitHub'dan indirilecek.

**Lokal kullanıyorsanız:** `real_drug_dataset.csv` dosyasını notebook ile aynı dizine yerleştirin.

In [ ]:
# Google Colab'da mı çalışıyoruz kontrol et
try:
    import google.colab
    IN_COLAB = True
    print("🌐 Google Colab ortamı tespit edildi!")
except:
    IN_COLAB = False
    print("💻 Lokal ortamda çalışıyorsunuz")

if IN_COLAB:
    import os
    
    # Veri seti zaten var mı kontrol et
    if os.path.exists('real_drug_dataset.csv'):
        print("✅ Veri seti zaten mevcut!")
    else:
        print("📥 Veri seti GitHub'dan indiriliyor...")
        print("   Repository: engindalgiC86-cpu/UU_VMIK_DRUGS")
        
        # GitHub'dan veri setini indir
        !wget -q https://raw.githubusercontent.com/engindalgiC86-cpu/UU_VMIK_DRUGS/main/real_drug_dataset.csv
        
        # İndirme başarılı mı kontrol et
        if os.path.exists('real_drug_dataset.csv'):
            file_size = os.path.getsize('real_drug_dataset.csv')
            print(f"   ✅ Veri seti başarıyla indirildi! ({file_size:,} bytes)")
        else:
            print("   ❌ HATA: Veri seti indirilemedi!")
            print("\n📝 Manuel yükleme adımları:")
            print("   1. Sol menüden 📁 Dosyalar sekmesini açın")
            print("   2. Upload butonuna tıklayın")
            print("   3. 'real_drug_dataset.csv' dosyasını yükleyin")
            raise FileNotFoundError("Veri seti bulunamadı!")
else:
    # Lokal ortam - dosya kontrolü
    import os
    if os.path.exists('real_drug_dataset.csv'):
        print("✅ Veri seti bulundu!")
    else:
        print("⚠️ UYARI: 'real_drug_dataset.csv' bulunamadı!")
        print("   Lütfen veri setini bu dizine yerleştirin.")

print("\n" + "="*70)
print("🚀 Kurulum tamamlandı! Veri yükleme işlemine geçebilirsiniz.")
print("="*70)

## Veri Setinin Yüklenmesi

In [ ]:
# Veri setini yükleme
df = pd.read_csv('real_drug_dataset.csv')

print(f"Veri seti boyutu: {df.shape}")
print(f"\nSatır sayısı: {df.shape[0]}")
print(f"Sütun sayısı: {df.shape[1]}")
print("\nİlk 5 satır:")
df.head()

In [ ]:
# Veri seti hakkında genel bilgi
print("Veri Seti Bilgileri:")
print("="*50)
df.info()

print("\nEksik Değerler:")
print("="*50)
print(df.isnull().sum())

print("\nSütun İsimleri:")
print("="*50)
print(df.columns.tolist())

---
## 1. Temel İstatistikler

Tüm sayısal sütunlar için temel istatistikleri hesaplayalım: mean (ortalama), std (standart sapma), min (minimum), %25 (1. çeyrek), median (medyan), %75 (3. çeyrek) ve max (maksimum).

In [ ]:
# Sayısal sütunları belirleme
numerical_columns = df.select_dtypes(include=[np.number]).columns.tolist()

print("Sayısal Sütunlar:")
print(numerical_columns)
print("\n" + "="*80)

# Temel istatistikleri hesaplama
stats_df = df[numerical_columns].describe().T
stats_df = stats_df[['mean', 'std', 'min', '25%', '50%', '75%', 'max']]
stats_df.columns = ['Ortalama', 'Std Sapma', 'Minimum', '%25', 'Medyan', '%75', 'Maksimum']

print("Sayısal Sütunlar İçin Temel İstatistikler:")
print("="*80)
stats_df.round(2)

### İstatistiklerin Yorumu

Yukarıdaki tabloda her sayısal değişken için:
- **Ortalama:** Verilerin merkezi eğilimi
- **Std Sapma:** Verilerin yayılımı (düşük değer = veriler ortalamaya yakın)
- **Minimum ve Maksimum:** Değer aralığı
- **%25, Medyan, %75:** Verilerin dağılımı hakkında bilgi verir

Medyan ile ortalama arasındaki fark, verilerin çarpıklığını gösterir.

---
## 2. Kategorik Sütunların Görselleştirilmesi

Kategorik sütunlardaki verileri her sütun için ayrı pasta grafikleri ile görselleştirelim.

In [ ]:
# Kategorik sütunları belirleme
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

print("Kategorik Sütunlar:")
print(categorical_columns)
print(f"\nToplam {len(categorical_columns)} kategorik sütun bulundu.")

In [ ]:
# Her kategorik sütun için pasta grafiği çizme
n_cols = 2  # Her satırda 2 grafik
n_rows = (len(categorical_columns) + 1) // 2

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axes = axes.flatten() if len(categorical_columns) > 1 else [axes]

for idx, col in enumerate(categorical_columns):
    # Her kategorinin sayısını hesaplama
    value_counts = df[col].value_counts()
    
    # Eğer çok fazla kategori varsa (>10), en yaygın 10'unu al
    if len(value_counts) > 10:
        value_counts = value_counts.head(10)
        title = f'{col}\n(En yaygın 10 kategori)'
    else:
        title = col
    
    # Pasta grafiği çizme
    axes[idx].pie(value_counts.values, labels=value_counts.index, autopct='%1.1f%%',
                  startangle=90, textprops={'fontsize': 9})
    axes[idx].set_title(title, fontsize=12, fontweight='bold', pad=20)

# Kullanılmayan eksenleri gizleme
for idx in range(len(categorical_columns), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('kategorik_sutunlar_pasta_grafikleri.png', dpi=300, bbox_inches='tight')
plt.show()

print("Pasta grafikleri başarıyla oluşturuldu!")

### Kategorik Değişkenlerin Detaylı İstatistikleri

In [ ]:
# Her kategorik sütun için frekans tablosu
for col in categorical_columns:
    print(f"\n{'='*60}")
    print(f"{col} - Frekans Dağılımı")
    print('='*60)
    
    freq_table = df[col].value_counts()
    freq_df = pd.DataFrame({
        'Kategori': freq_table.index,
        'Frekans': freq_table.values,
        'Yüzde': (freq_table.values / len(df) * 100).round(2)
    })
    
    if len(freq_df) > 15:
        print(freq_df.head(15))
        print(f"\n... ve {len(freq_df) - 15} kategori daha")
    else:
        print(freq_df)
    
    print(f"\nBenzersiz kategori sayısı: {df[col].nunique()}")

---
## 3. Hastalık Sebepleri ve Tedavi Süresi Dağılımı

Condition (hastalık) sütunundaki sebeplere karşılık gelen tedavi süresi (Treatment_Duration_days) dağılımını boxplot ile görselleştirelim.

In [ ]:
# Condition ve Treatment_Duration_days sütunlarının varlığını kontrol et
if 'Condition' in df.columns and 'Treatment_Duration_days' in df.columns:
    
    # En yaygın 15 hastalığı seç (görselleştirmeyi daha okunabilir yapmak için)
    top_conditions = df['Condition'].value_counts().head(15).index
    df_filtered = df[df['Condition'].isin(top_conditions)]
    
    # Boxplot oluşturma
    plt.figure(figsize=(16, 8))
    
    # Seaborn boxplot
    sns.boxplot(data=df_filtered, x='Condition', y='Treatment_Duration_days', 
                palette='Set2', linewidth=1.5)
    
    plt.title('Hastalık Türlerine Göre Tedavi Süresi Dağılımı\n(En Yaygın 15 Hastalık)', 
              fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Hastalık Türü', fontsize=13, fontweight='bold')
    plt.ylabel('Tedavi Süresi (Gün)', fontsize=13, fontweight='bold')
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.grid(axis='y', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig('hastalik_tedavi_suresi_boxplot.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # İstatistiksel özet
    print("\nHastalık Türlerine Göre Tedavi Süresi İstatistikleri:")
    print("="*80)
    summary_stats = df_filtered.groupby('Condition')['Treatment_Duration_days'].agg([
        ('Ortalama', 'mean'),
        ('Medyan', 'median'),
        ('Std Sapma', 'std'),
        ('Min', 'min'),
        ('Max', 'max'),
        ('Vaka Sayısı', 'count')
    ]).round(2).sort_values('Ortalama', ascending=False)
    
    print(summary_stats)
    
else:
    print("Uyarı: 'Condition' veya 'Treatment_Duration_days' sütunu bulunamadı.")
    print("Mevcut sütunlar:", df.columns.tolist())

### Boxplot Yorumu

**Boxplot Elemanları:**
- **Kutu:** Verilerin %50'sini içerir (1. ve 3. çeyrek arası)
- **Ortadaki çizgi:** Medyan (ortanca değer)
- **Bıyıklar:** Minimum ve maksimum değerler (aykırı değerler hariç)
- **Noktalar:** Aykırı değerler (outliers)

**Gözlemler:**
- Farklı hastalıklar için tedavi sürelerinin nasıl değiştiğini görebiliriz
- Bazı hastalıkların daha uzun ve değişken tedavi süreleri olabilir
- Aykırı değerler, normal olmayan uzun veya kısa tedavi sürelerini gösterir

---
## 4. İlaç İsimleri ve Yan Etkiler - 2D Histogram

Drug_Name ve Side_Effects sütunları için 2 boyutlu histogram (2D frekans görselleştirmesi) çizelim.

In [ ]:
# Drug_Name ve Side_Effects sütunlarını kontrol et
if 'Drug_Name' in df.columns and 'Side_Effects' in df.columns:
    
    # En yaygın 20 ilacı ve 20 yan etkiyi seç
    top_drugs = df['Drug_Name'].value_counts().head(20).index
    top_side_effects = df['Side_Effects'].value_counts().head(20).index
    
    # Filtrelenmiş veri
    df_filtered = df[(df['Drug_Name'].isin(top_drugs)) & 
                     (df['Side_Effects'].isin(top_side_effects))]
    
    # Frekans tablosu oluşturma (2D)
    freq_table = pd.crosstab(df_filtered['Drug_Name'], df_filtered['Side_Effects'])
    
    # 2D Heatmap (ısı haritası) ile görselleştirme
    plt.figure(figsize=(18, 12))
    
    sns.heatmap(freq_table, annot=True, fmt='d', cmap='YlOrRd', 
                linewidths=0.5, cbar_kws={'label': 'Frekans'})
    
    plt.title('İlaç İsimleri ve Yan Etkiler - 2D Frekans Haritası\n(En Yaygın 20 İlaç ve 20 Yan Etki)', 
              fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Yan Etkiler', fontsize=13, fontweight='bold')
    plt.ylabel('İlaç İsimleri', fontsize=13, fontweight='bold')
    plt.xticks(rotation=45, ha='right', fontsize=9)
    plt.yticks(rotation=0, fontsize=9)
    
    plt.tight_layout()
    plt.savefig('ilac_yanetki_2d_histogram.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n2D Histogram Özet İstatistikleri:")
    print("="*60)
    print(f"Analiz edilen benzersiz ilaç sayısı: {len(top_drugs)}")
    print(f"Analiz edilen benzersiz yan etki sayısı: {len(top_side_effects)}")
    print(f"Toplam vaka sayısı: {len(df_filtered)}")
    
    # En yaygın ilaç-yan etki kombinasyonları
    print("\nEn Yaygın İlaç-Yan Etki Kombinasyonları:")
    print("="*60)
    
    # Kombinasyon frekanslarını düzleştir ve sırala
    combinations = []
    for drug in freq_table.index:
        for side_effect in freq_table.columns:
            count = freq_table.loc[drug, side_effect]
            if count > 0:
                combinations.append((drug, side_effect, count))
    
    # En yaygın 15 kombinasyonu göster
    combinations_sorted = sorted(combinations, key=lambda x: x[2], reverse=True)[:15]
    
    combo_df = pd.DataFrame(combinations_sorted, 
                            columns=['İlaç', 'Yan Etki', 'Frekans'])
    combo_df.index = range(1, len(combo_df) + 1)
    print(combo_df)
    
else:
    print("Uyarı: 'Drug_Name' veya 'Side_Effects' sütunu bulunamadı.")
    print("Mevcut sütunlar:", df.columns.tolist())

### 2D Histogram Yorumu

**Isı Haritası (Heatmap) Analizi:**

- **Koyu renkli hücreler:** İlgili ilaç-yan etki kombinasyonunun sık görüldüğünü gösterir
- **Açık renkli hücreler:** Daha az sıklıkla görülen kombinasyonlar
- **Beyaz hücreler:** Hiç görülmeyen veya çok nadir görülen kombinasyonlar

**Çıkarımlar:**
1. Belirli ilaçların belirli yan etkilerle daha sık ilişkili olduğunu görebiliriz
2. Bazı yan etkiler birçok farklı ilaçta yaygın olabilir
3. Bazı ilaçlar daha fazla çeşitlilikte yan etki gösterebilir
4. Bu bilgi, ilaç güvenliği ve hasta bilgilendirmesi için önemlidir

---
## 5. Hiyerarşik Kümeleme Analizi

4. maddedeki analizde kullandığımız veriler için, seaborn kütüphanesinden clustermap fonksiyonunu kullanarak hiyerarşik kümeleme yapalım.

In [ ]:
# Drug_Name ve Side_Effects sütunlarını kontrol et
if 'Drug_Name' in df.columns and 'Side_Effects' in df.columns:
    
    # En yaygın 25 ilacı ve 25 yan etkiyi seç (daha detaylı analiz için)
    top_drugs_cluster = df['Drug_Name'].value_counts().head(25).index
    top_side_effects_cluster = df['Side_Effects'].value_counts().head(25).index
    
    # Filtrelenmiş veri
    df_cluster = df[(df['Drug_Name'].isin(top_drugs_cluster)) & 
                    (df['Side_Effects'].isin(top_side_effects_cluster))]
    
    # Frekans tablosu oluşturma
    freq_table_cluster = pd.crosstab(df_cluster['Drug_Name'], 
                                     df_cluster['Side_Effects'])
    
    # Normalizasyon (her ilaç için yan etki oranları)
    freq_table_normalized = freq_table_cluster.div(freq_table_cluster.sum(axis=1), axis=0)
    
    print("Hiyerarşik Kümeleme Analizi Başlatılıyor...")
    print("="*60)
    
    # Clustermap oluşturma
    g = sns.clustermap(freq_table_normalized, 
                      cmap='RdYlGn_r',  # Renk paleti
                      figsize=(20, 16),
                      linewidths=0.5,
                      cbar_kws={'label': 'Normalize Edilmiş Frekans'},
                      method='ward',  # Kümeleme metodu
                      metric='euclidean',  # Mesafe metriği
                      dendrogram_ratio=0.15,
                      cbar_pos=(0.02, 0.83, 0.03, 0.15))
    
    g.fig.suptitle('İlaç-Yan Etki Hiyerarşik Kümeleme Analizi\n(Normalize Edilmiş Frekanslar)', 
                   fontsize=18, fontweight='bold', y=0.98)
    
    g.ax_heatmap.set_xlabel('Yan Etkiler', fontsize=13, fontweight='bold')
    g.ax_heatmap.set_ylabel('İlaç İsimleri', fontsize=13, fontweight='bold')
    
    plt.setp(g.ax_heatmap.get_xticklabels(), rotation=45, ha='right', fontsize=9)
    plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0, fontsize=9)
    
    plt.savefig('hiyerarsik_kumeleme_clustermap.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\nHiyerarşik Kümeleme tamamlandı!")
    
else:
    print("Uyarı: Gerekli sütunlar bulunamadı.")

### Hiyerarşik Kümeleme Analizi Yorumu

**Clustermap Nedir?**

Clustermap, hem satırları (ilaçlar) hem de sütunları (yan etkiler) benzerliklerine göre gruplandırır ve dendrogramlar (ağaç diyagramları) ile görselleştirir.

**Dendrogramlar (Ağaç Diyagramları):**
- **Sol taraftaki dendogram:** İlaçları benzer yan etki profillerine göre gruplar
- **Üst taraftaki dendogram:** Yan etkileri birlikte görüldükleri ilaçlara göre gruplar
- **Dal uzunlukları:** Gruplar arasındaki farklılığı gösterir (uzun dal = daha farklı)

**Renkler:**
- **Kırmızı/Koyu renkler:** Yüksek frekans (bu ilaç için bu yan etki yaygın)
- **Yeşil/Açık renkler:** Düşük frekans (bu yan etki bu ilaç için nadir)

**Çıkarımlar:**

1. **İlaç Grupları:** Benzer yan etki profillerine sahip ilaçlar bir arada kümelenir
   - Aynı terapötik sınıftan ilaçlar benzer yan etki profilleri gösterebilir
   - Bu bilgi, ilaç değiştirme kararlarında yardımcı olabilir

2. **Yan Etki Grupları:** Birlikte sıklıkla görülen yan etkiler gruplar halinde olur
   - Bazı yan etkiler birlikte ortaya çıkma eğilimindedir
   - Bu ilişkiler farmakolojik mekanizmalarla açıklanabilir

3. **Özel Durumlar:**
   - Tek başına duran ilaçlar benzersiz yan etki profilleri gösterir
   - Bu ilaçlar özel dikkat gerektirebilir

4. **Klinik Önem:**
   - Hastalara ilaç reçete ederken alternatif seçenekler belirlenebilir
   - Yan etki riski yüksek olan hasta grupları için daha güvenli seçenekler önerilebilir

---
## 6. Yaş ve İyileşme Skoru Korelasyonu

Yaş (Age) ve Improvement_Score değişkenleri arasındaki korelasyonu hesaplayalım ve yorumlayalım.

In [ ]:
# Age ve Improvement_Score sütunlarını kontrol et
if 'Age' in df.columns and 'Improvement_Score' in df.columns:
    
    # Eksik değerleri çıkar
    df_corr = df[['Age', 'Improvement_Score']].dropna()
    
    # Pearson korelasyon katsayısı
    pearson_corr = df_corr['Age'].corr(df_corr['Improvement_Score'])
    
    # Spearman korelasyon katsayısı (sıralama bazlı, outlier'lara daha dayanıklı)
    spearman_corr = df_corr['Age'].corr(df_corr['Improvement_Score'], method='spearman')
    
    # İstatistiksel anlamlılık testi
    from scipy.stats import pearsonr, spearmanr
    
    pearson_stat, pearson_pvalue = pearsonr(df_corr['Age'], df_corr['Improvement_Score'])
    spearman_stat, spearman_pvalue = spearmanr(df_corr['Age'], df_corr['Improvement_Score'])
    
    print("YAŞ ve İYİLEŞME SKORU KORELASYON ANALİZİ")
    print("="*70)
    print(f"\nVeri Sayısı: {len(df_corr)}")
    print("\nPearson Korelasyon Katsayısı: {:.4f}".format(pearson_corr))
    print(f"P-değeri: {pearson_pvalue:.4e}")
    print(f"İstatistiksel Anlamlılık: {'Evet' if pearson_pvalue < 0.05 else 'Hayır'} (α=0.05)")
    
    print("\nSpearman Korelasyon Katsayısı: {:.4f}".format(spearman_corr))
    print(f"P-değeri: {spearman_pvalue:.4e}")
    print(f"İstatistiksel Anlamlılık: {'Evet' if spearman_pvalue < 0.05 else 'Hayır'} (α=0.05)")
    
    # Görselleştirme
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Scatter plot
    axes[0].scatter(df_corr['Age'], df_corr['Improvement_Score'], 
                   alpha=0.5, s=30, color='steelblue', edgecolors='black', linewidth=0.5)
    
    # Regresyon çizgisi ekleme
    z = np.polyfit(df_corr['Age'], df_corr['Improvement_Score'], 1)
    p = np.poly1d(z)
    axes[0].plot(df_corr['Age'], p(df_corr['Age']), "r--", linewidth=2, 
                label=f'Regresyon Doğrusu\nr = {pearson_corr:.4f}')
    
    axes[0].set_xlabel('Yaş', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('İyileşme Skoru', fontsize=12, fontweight='bold')
    axes[0].set_title('Yaş vs İyileşme Skoru\nScatter Plot', 
                     fontsize=14, fontweight='bold', pad=15)
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3, linestyle='--')
    
    # Korelasyon matrisi heatmap
    corr_matrix = df_corr.corr()
    sns.heatmap(corr_matrix, annot=True, fmt='.4f', cmap='coolwarm', 
               center=0, square=True, linewidths=2, cbar_kws={'label': 'Korelasyon'},
               ax=axes[1], vmin=-1, vmax=1)
    axes[1].set_title('Korelasyon Matrisi', fontsize=14, fontweight='bold', pad=15)
    
    plt.tight_layout()
    plt.savefig('yas_iyilesme_korelasyon.png', dpi=300, bbox_inches='tight')
    plt.show()
    
else:
    print("Uyarı: 'Age' veya 'Improvement_Score' sütunu bulunamadı.")
    print("Mevcut sütunlar:", df.columns.tolist())

### Korelasyon Analizi Yorumu

**Korelasyon Katsayısı Değerlendirmesi:**

Korelasyon katsayısı -1 ile +1 arasında değer alır:
- **+1:** Mükemmel pozitif korelasyon (bir değişken arttıkça diğeri de artar)
- **0:** Korelasyon yok (değişkenler arasında doğrusal ilişki yok)
- **-1:** Mükemmel negatif korelasyon (bir değişken arttıkça diğeri azalır)

**Genel Değerlendirme Kriterleri:**
- |r| < 0.3: Zayıf korelasyon
- 0.3 ≤ |r| < 0.7: Orta düzey korelasyon
- |r| ≥ 0.7: Güçlü korelasyon

**İstatistiksel Anlamlılık (P-değeri):**
- p < 0.05: İstatistiksel olarak anlamlı (korelasyon tesadüfi değil)
- p ≥ 0.05: İstatistiksel olarak anlamlı değil (korelasyon tesadüfi olabilir)

**Pearson vs Spearman:**
- **Pearson:** Doğrusal ilişkileri ölçer, normal dağılıma duyarlı
- **Spearman:** Monoton ilişkileri ölçer, aykırı değerlere daha dayanıklı

**Klinik/Pratik Yorumlama:**

Yukarıdaki sonuçlara göre:
- Eğer pozitif korelasyon varsa: Yaşlı hastalar daha yüksek iyileşme skoru gösterebilir (veya tersi)
- Eğer negatif korelasyon varsa: Genç hastalar daha iyi iyileşme gösterebilir (veya tersi)
- Eğer korelasyon zayıfsa: Yaşın iyileşme üzerinde doğrudan etkisi sınırlıdır

**ÖNEMLİ NOT:** Korelasyon nedensellik anlamına gelmez! Başka faktörler (hastalık türü, ilaç, genel sağlık durumu vb.) de etkili olabilir.

---
## 7. Cinsiyet Bazında Hastalık Oranları

Erkekler ve kadınlar için hastalık (Condition) oranlarını normalize edilmiş bir şekilde hesaplayalım.

In [ ]:
# Gender ve Condition sütunlarını kontrol et (veya Sex/Gender gibi farklı isimler olabilir)
gender_col = None
for col in ['Gender', 'Sex', 'gender', 'sex']:
    if col in df.columns:
        gender_col = col
        break

if gender_col and 'Condition' in df.columns:
    
    # Cinsiyet ve hastalık verilerini filtrele (eksik değerleri çıkar)
    df_gender = df[[gender_col, 'Condition']].dropna()
    
    print("CİNSİYET BAZINDA HASTALIK ORANLARI ANALİZİ")
    print("="*70)
    
    # Her cinsiyet için hastalık sayıları
    gender_condition_counts = pd.crosstab(df_gender[gender_col], df_gender['Condition'])
    
    # Normalize etme (her cinsiyet için toplam 1 olacak şekilde)
    gender_condition_normalized = gender_condition_counts.div(
        gender_condition_counts.sum(axis=1), axis=0
    )
    
    print("\nNormalize Edilmiş Hastalık Oranları (Her cinsiyet için toplam = 1.0):")
    print("="*70)
    
    # Tüm hastalıklar için normalize oranları göster
    display_df = gender_condition_normalized.T
    display_df = display_df.sort_values(by=display_df.columns[0], ascending=False)
    
    print(display_df.round(4))
    
    # Her cinsiyet için en yaygın 10 hastalık
    print("\n" + "="*70)
    for gender in gender_condition_normalized.index:
        print(f"\n{gender} için En Yaygın 10 Hastalık:")
        print("-" * 60)
        top_conditions = gender_condition_normalized.loc[gender].sort_values(ascending=False).head(10)
        
        for i, (condition, prob) in enumerate(top_conditions.items(), 1):
            print(f"{i:2d}. {condition:30s} - {prob:.4f} ({prob*100:.2f}%)")
    
    # Görselleştirme - En yaygın 15 hastalık için
    top_15_conditions = df_gender['Condition'].value_counts().head(15).index
    plot_data = gender_condition_normalized[top_15_conditions]
    
    fig, axes = plt.subplots(2, 1, figsize=(16, 12))
    
    # Grouped bar chart
    plot_data.T.plot(kind='bar', ax=axes[0], width=0.8, edgecolor='black', linewidth=0.7)
    axes[0].set_title('Cinsiyet Bazında Hastalık Oranları - Karşılaştırma\n(En Yaygın 15 Hastalık)', 
                     fontsize=14, fontweight='bold', pad=15)
    axes[0].set_xlabel('Hastalık', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Olasılık (Normalize)', fontsize=12, fontweight='bold')
    axes[0].legend(title='Cinsiyet', fontsize=10)
    axes[0].grid(axis='y', alpha=0.3, linestyle='--')
    axes[0].tick_params(axis='x', rotation=45)
    plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    # Heatmap
    sns.heatmap(plot_data.T, annot=True, fmt='.3f', cmap='YlOrRd', 
               linewidths=0.5, cbar_kws={'label': 'Olasılık'}, ax=axes[1])
    axes[1].set_title('Cinsiyet-Hastalık Olasılık Matrisi', 
                     fontsize=14, fontweight='bold', pad=15)
    axes[1].set_xlabel('Cinsiyet', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Hastalık', fontsize=12, fontweight='bold')
    plt.setp(axes[1].yaxis.get_majorticklabels(), rotation=0)
    
    plt.tight_layout()
    plt.savefig('cinsiyet_hastalik_oranlari.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # İstatistiksel karşılaştırma
    print("\n" + "="*70)
    print("İstatistiksel Özet:")
    print("="*70)
    
    for gender in gender_condition_normalized.index:
        total_cases = gender_condition_counts.loc[gender].sum()
        unique_conditions = (gender_condition_counts.loc[gender] > 0).sum()
        print(f"\n{gender}:")
        print(f"  - Toplam vaka sayısı: {total_cases}")
        print(f"  - Benzersiz hastalık sayısı: {unique_conditions}")
        print(f"  - Ortalama hastalık olasılığı: {gender_condition_normalized.loc[gender].mean():.4f}")
    
else:
    print("Uyarı: Cinsiyet veya 'Condition' sütunu bulunamadı.")
    print("Mevcut sütunlar:", df.columns.tolist())

### Cinsiyet Bazında Hastalık Oranları Yorumu

**Normalizasyon Nedir?**

Her cinsiyet için hastalık oranlarını toplam 1 olacak şekilde normalize ettik. Bu sayede:
- Farklı cinsiyet gruplarındaki toplam vaka sayıları eşit olmasa bile adil karşılaştırma yapabiliriz
- Her değer, o cinsiyetteki toplam vakaların yüzde kaçının o hastalık olduğunu gösterir

**Yorumlama:**

1. **Hastalık Dağılımı Farklılıkları:**
   - Bazı hastalıklar bir cinsiyette daha yaygın olabilir
   - Bu farklılıklar biyolojik, hormonal veya yaşam tarzı faktörlerinden kaynaklanabilir

2. **Yüksek Olasılık:**
   - Bir hastalığın normalize olasılığı yüksekse, o hastalık o cinsiyet için daha yaygın demektir
   - Örnek: Kadınlarda 0.15 değeri = bu hastalık kadın vakalarının %15'ini oluşturuyor

3. **Cinsiyet Karşılaştırması:**
   - Grafikte çubuklar arasındaki farklar, hastalığın cinsiyetler arası dağılım farklılıklarını gösterir
   - Büyük farklar, hastalığın cinsiyet-spesifik olabileceğini düşündürür

4. **Klinik Önem:**
   - Bu bilgi, risk değerlendirmesi ve erken tanı için kullanılabilir
   - Cinsiyet-spesifik tarama programları planlanabilir
   - Hasta eğitimi ve bilinçlendirme çalışmaları yönlendirilebilir

---
## 8. Baş Dönmesi (Dizziness) Yan Etkisi Raporlanan İlaçlar

Hangi ilaçlar için baş dönmesi yan etkisi raporlanmıştır? Bu ilaçları azalan sıklığa göre sıralayalım.

In [ ]:
# Drug_Name ve Side_Effects sütunlarını kontrol et
if 'Drug_Name' in df.columns and 'Side_Effects' in df.columns:
    
    print("BAŞ DÖNMESİ (DIZZINESS) YAN ETKİSİ ANALİZİ")
    print("="*70)
    
    # Baş dönmesi yan etkisi olan vakaları filtrele
    # Side_Effects sütununda 'Dizziness', 'dizziness', veya benzer ifadeler olabilir
    dizziness_cases = df[df['Side_Effects'].str.contains('Dizziness|dizziness', 
                                                         case=False, 
                                                         na=False)]
    
    if len(dizziness_cases) > 0:
        # İlaçları baş dönmesi sıklığına göre say ve sırala
        dizziness_drugs = dizziness_cases['Drug_Name'].value_counts()
        
        print(f"\nBaş dönmesi yan etkisi raporlanan toplam vaka sayısı: {len(dizziness_cases)}")
        print(f"Baş dönmesi yan etkisi olan benzersiz ilaç sayısı: {len(dizziness_drugs)}")
        print(f"\nToplam veri setindeki vaka sayısı: {len(df)}")
        print(f"Baş dönmesi oranı: {len(dizziness_cases)/len(df)*100:.2f}%")
        
        print("\n" + "="*70)
        print("BAŞ DÖNMESİ RAPORLANAN İLAÇLAR (Azalan Sıklığa Göre):")
        print("="*70)
        print(f"\n{'Sıra':<6} {'İlaç İsmi':<35} {'Vaka Sayısı':<12} {'Yüzde':<10}")
        print("-" * 70)
        
        # Tüm ilaçları listele
        for rank, (drug, count) in enumerate(dizziness_drugs.items(), 1):
            percentage = (count / len(dizziness_cases)) * 100
            print(f"{rank:<6} {drug:<35} {count:<12} {percentage:>6.2f}%")
        
        # DataFrame olarak göster (ilk 30 ilaç)
        print("\n" + "="*70)
        print("DETAYLI TABLO (İlk 30 İlaç):")
        print("="*70)
        
        dizziness_df = pd.DataFrame({
            'İlaç İsmi': dizziness_drugs.index,
            'Baş Dönmesi Vaka Sayısı': dizziness_drugs.values,
            'Toplam İçindeki Yüzde (%)': (dizziness_drugs.values / len(dizziness_cases) * 100).round(2)
        })
        
        dizziness_df.index = range(1, len(dizziness_df) + 1)
        print(dizziness_df.head(30))
        
        # Görselleştirme - En yaygın 20 ilaç
        plt.figure(figsize=(16, 10))
        
        top_20 = dizziness_drugs.head(20)
        
        bars = plt.barh(range(len(top_20)), top_20.values, color='coral', 
                       edgecolor='black', linewidth=1.2)
        
        plt.yticks(range(len(top_20)), top_20.index, fontsize=11)
        plt.xlabel('Baş Dönmesi Vaka Sayısı', fontsize=13, fontweight='bold')
        plt.ylabel('İlaç İsmi', fontsize=13, fontweight='bold')
        plt.title('Baş Dönmesi Yan Etkisi En Çok Raporlanan 20 İlaç', 
                 fontsize=15, fontweight='bold', pad=20)
        plt.grid(axis='x', alpha=0.3, linestyle='--')
        
        # Çubukların üzerine değerleri yaz
        for i, (bar, value) in enumerate(zip(bars, top_20.values)):
            plt.text(value + 0.5, bar.get_y() + bar.get_height()/2, 
                    f'{value}', va='center', fontsize=10, fontweight='bold')
        
        plt.gca().invert_yaxis()  # En yüksek değer üstte olsun
        plt.tight_layout()
        plt.savefig('bas_donmesi_ilaclar.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # Pasta grafiği - En yaygın 10 ilaç
        plt.figure(figsize=(12, 8))
        
        top_10 = dizziness_drugs.head(10)
        other_count = dizziness_drugs[10:].sum()
        
        if other_count > 0:
            plot_data = pd.concat([top_10, pd.Series({'Diğerleri': other_count})])
        else:
            plot_data = top_10
        
        plt.pie(plot_data.values, labels=plot_data.index, autopct='%1.1f%%',
               startangle=90, textprops={'fontsize': 10})
        plt.title('Baş Dönmesi Vakalarının İlaçlara Göre Dağılımı\n(En Yaygın 10 İlaç + Diğerleri)', 
                 fontsize=14, fontweight='bold', pad=20)
        plt.axis('equal')
        
        plt.tight_layout()
        plt.savefig('bas_donmesi_dagilim_pasta.png', dpi=300, bbox_inches='tight')
        plt.show()
        
    else:
        print("\nUyarı: Veri setinde 'Dizziness' yan etkisi bulunamadı.")
        print("Mevcut yan etkiler örneği:")
        print(df['Side_Effects'].value_counts().head(10))
        
else:
    print("Uyarı: 'Drug_Name' veya 'Side_Effects' sütunu bulunamadı.")
    print("Mevcut sütunlar:", df.columns.tolist())

### Baş Dönmesi Yan Etkisi Analizi Yorumu

**Bulgular:**

1. **Yaygınlık:** Tabloda hangi ilaçların baş dönmesi yan etkisini daha sık rapor ettiğini görebiliriz

2. **Risk Değerlendirmesi:**
   - Listede üst sıralarda olan ilaçlar, baş dönmesi riski daha yüksek ilaçlardır
   - Hasta güvenliği açısından bu bilgi önemlidir

3. **Klinik Uygulamalar:**
   - Yüksek baş dönmesi riski olan hastalara (örn. yaşlılar, düşme riski olanlar) bu ilaçlar reçete edilirken dikkatli olunmalı
   - Hastalar baş dönmesi riski konusunda bilgilendirilmeli
   - Gerekirse alternatif ilaçlar değerlendirilmeli

4. **Farmakovijilans:** Bu tür analizler, ilaç güvenliği takibi için önemlidir

---
## 9. Metoprolol İlacı İçin Baş Dönmesi Olasılığı

Metoprolol ilacı için, diğer yan etkilere göre baş dönmesi yan etkisinin olasılığını hesaplayalım.

In [ ]:
# Drug_Name ve Side_Effects sütunlarını kontrol et
if 'Drug_Name' in df.columns and 'Side_Effects' in df.columns:
    
    print("METOPROLOL İÇİN BAŞ DÖNMESİ OLASILIĞI ANALİZİ")
    print("="*70)
    
    # Metoprolol vakalarını filtrele
    metoprolol_cases = df[df['Drug_Name'].str.contains('Metoprolol|metoprolol', 
                                                       case=False, 
                                                       na=False)]
    
    if len(metoprolol_cases) > 0:
        print(f"\nMetoprolol toplam vaka sayısı: {len(metoprolol_cases)}")
        
        # Metoprolol için tüm yan etkiler
        all_side_effects = metoprolol_cases['Side_Effects'].value_counts()
        
        print(f"\nMetoprolol için raporlanan benzersiz yan etki sayısı: {len(all_side_effects)}")
        
        # Baş dönmesi vakalarını say
        dizziness_count = metoprolol_cases[
            metoprolol_cases['Side_Effects'].str.contains('Dizziness|dizziness', 
                                                          case=False, 
                                                          na=False)
        ].shape[0]
        
        # Olasılık hesaplama
        total_metoprolol = len(metoprolol_cases)
        dizziness_probability = dizziness_count / total_metoprolol if total_metoprolol > 0 else 0
        
        print("\n" + "="*70)
        print("BAŞ DÖNMESİ OLASILIĞI HESAPLAMA:")
        print("="*70)
        print(f"\nMetoprolol toplam vaka sayısı: {total_metoprolol}")
        print(f"Baş dönmesi rapor edilen vaka sayısı: {dizziness_count}")
        print(f"\nBaş Dönmesi Olasılığı: {dizziness_probability:.4f}")
        print(f"Baş Dönmesi Yüzdesi: {dizziness_probability * 100:.2f}%")
        
        print("\n" + "="*70)
        print("METOPROLOL İÇİN TÜM YAN ETKİLER (Sıklığa Göre):")
        print("="*70)
        
        # Yan etki dağılımını göster
        side_effects_df = pd.DataFrame({
            'Yan Etki': all_side_effects.index,
            'Vaka Sayısı': all_side_effects.values,
            'Olasılık': (all_side_effects.values / total_metoprolol).round(4),
            'Yüzde (%)': (all_side_effects.values / total_metoprolol * 100).round(2)
        })
        
        side_effects_df.index = range(1, len(side_effects_df) + 1)
        print(side_effects_df)
        
        # Baş dönmesini vurgula
        dizziness_rows = side_effects_df[
            side_effects_df['Yan Etki'].str.contains('Dizziness|dizziness', case=False, na=False)
        ]
        
        if len(dizziness_rows) > 0:
            print("\n" + "="*70)
            print("BAŞ DÖNMESİ DETAYLARI:")
            print("="*70)
            print(dizziness_rows)
        
        # Görselleştirme
        fig, axes = plt.subplots(1, 2, figsize=(18, 7))
        
        # Bar chart - En yaygın 15 yan etki
        top_15_effects = all_side_effects.head(15)
        colors = ['red' if 'dizziness' in str(effect).lower() else 'steelblue' 
                 for effect in top_15_effects.index]
        
        bars = axes[0].barh(range(len(top_15_effects)), top_15_effects.values, 
                           color=colors, edgecolor='black', linewidth=1)
        axes[0].set_yticks(range(len(top_15_effects)))
        axes[0].set_yticklabels(top_15_effects.index, fontsize=10)
        axes[0].set_xlabel('Vaka Sayısı', fontsize=12, fontweight='bold')
        axes[0].set_title('Metoprolol - En Yaygın 15 Yan Etki\n(Kırmızı: Baş Dönmesi)', 
                         fontsize=13, fontweight='bold', pad=15)
        axes[0].grid(axis='x', alpha=0.3, linestyle='--')
        axes[0].invert_yaxis()
        
        # Değerleri çubukların üzerine yaz
        for i, (bar, value) in enumerate(zip(bars, top_15_effects.values)):
            axes[0].text(value + 0.5, bar.get_y() + bar.get_height()/2, 
                        f'{value}', va='center', fontsize=9, fontweight='bold')
        
        # Pasta grafiği - Baş dönmesi vs Diğerleri
        pie_data = pd.Series({
            'Baş Dönmesi': dizziness_count,
            'Diğer Yan Etkiler': total_metoprolol - dizziness_count
        })
        
        colors_pie = ['red', 'lightblue']
        explode = (0.1, 0)  # Baş dönmesini vurgula
        
        axes[1].pie(pie_data.values, labels=pie_data.index, autopct='%1.2f%%',
                   colors=colors_pie, explode=explode, startangle=90,
                   textprops={'fontsize': 12, 'fontweight': 'bold'},
                   shadow=True)
        axes[1].set_title('Metoprolol - Baş Dönmesi Oranı', 
                         fontsize=13, fontweight='bold', pad=15)
        
        plt.tight_layout()
        plt.savefig('metoprolol_bas_donmesi_analizi.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # İstatistiksel özet
        print("\n" + "="*70)
        print("ÖZETİN ÖZETİ:")
        print("="*70)
        print(f"\n✓ Metoprolol kullanan {total_metoprolol} hastadan")
        print(f"  {dizziness_count} tanesi ({dizziness_probability*100:.2f}%) baş dönmesi yaşadı.")
        print(f"\n✓ Yani, Metoprolol kullanan her 100 hastadan")
        print(f"  yaklaşık {int(dizziness_probability*100)} tanesi baş dönmesi rapor etti.")
        
        if dizziness_probability > 0.1:
            print("\n⚠ DİKKAT: Baş dönmesi oranı %10'un üzerinde - Yüksek risk!")
        elif dizziness_probability > 0.05:
            print("\n⚡ UYARI: Baş dönmesi oranı orta düzeyde - Dikkat gerektirir.")
        else:
            print("\n✓ Baş dönmesi oranı düşük seviyede.")
            
    else:
        print("\nUyarı: Veri setinde 'Metoprolol' ilacı bulunamadı.")
        print("\nMevcut ilaçlar örneği:")
        print(df['Drug_Name'].value_counts().head(20))
        
else:
    print("Uyarı: 'Drug_Name' veya 'Side_Effects' sütunu bulunamadı.")
    print("Mevcut sütunlar:", df.columns.tolist())

### Metoprolol Baş Dönmesi Olasılığı Yorumu

**Olasılık Hesaplaması:**

$$P(\text{Baş Dönmesi | Metoprolol}) = \frac{\text{Metoprolol ile baş dönmesi vaka sayısı}}{\text{Toplam Metoprolol vaka sayısı}}$$

**Klinik Yorumlama:**

1. **Risk Değerlendirmesi:**
   - Hesaplanan olasılık, Metoprolol kullanan hastalarda baş dönmesi görülme riskini gösterir
   - Bu bilgi, ilaç reçete ederken risk-fayda değerlendirmesi için kullanılır

2. **Hasta Bilgilendirmesi:**
   - Hastalar bu yan etki riski konusunda bilgilendirilmelidir
   - Özellikle araç kullanan veya yüksekte çalışan hastalar için önemlidir

3. **Karşılaştırma:**
   - Bu olasılık, diğer beta-blokerler veya alternatif ilaçlarla karşılaştırılabilir
   - Daha düşük baş dönmesi riski olan alternatifler tercih edilebilir

4. **Metoprolol Hakkında:**
   - Metoprolol yaygın olarak kullanılan bir beta-blokerdir
   - Hipertansiyon, anjina ve kalp yetmezliği tedavisinde kullanılır
   - Baş dönmesi, beta-blokerlerin bilinen yan etkilerinden biridir (kan basıncı düşürücü etki nedeniyle)

**Pratik Öneriler:**
- İlaç başlangıcında düşük dozla başlanmalı
- Hastalar yavaşça ayağa kalkma konusunda uyarılmalı
- Baş dönmesi şiddetli veya sürekli ise doktora başvurulmalı

---
## Genel Sonuçlar ve Değerlendirme

Bu projede, ilaçlar ve yan etkileri veri seti üzerinde kapsamlı bir veri madenciliği analizi gerçekleştirdik.

**Ana Bulgular:**

1. **İstatistiksel Analiz:** Sayısal değişkenler için temel istatistikler hesaplandı
2. **Kategorik Analiz:** Kategorik değişkenlerin dağılımları görselleştirildi
3. **Hastalık-Tedavi İlişkisi:** Hastalık türleri ile tedavi süreleri arasındaki ilişki incelendi
4. **İlaç-Yan Etki İlişkisi:** 2D histogram ve hiyerarşik kümeleme ile ilaç-yan etki örüntüleri belirlendi
5. **Korelasyon Analizi:** Yaş ve iyileşme skoru arasındaki ilişki değerlendirildi
6. **Cinsiyet Analizi:** Hastalıkların cinsiyete göre dağılımı normalize edilerek karşılaştırıldı
7. **Yan Etki Analizi:** Baş dönmesi yan etkisi detaylı olarak incelendi
8. **İlaç-Spesifik Analiz:** Metoprolol için yan etki profili çıkarıldı

**Metodolojik Yaklaşımlar:**
- Tanımlayıcı istatistikler
- Görselleştirme teknikleri (pasta grafikleri, boxplot, histogram, heatmap)
- Hiyerarşik kümeleme
- Korelasyon analizi
- Normalizasyon ve olasılık hesaplamaları

**Veri Kalitesi Notları:**
- Eksik değerler kontrol edildi
- Veri tipleri doğrulandı
- Outlier'lar görselleştirildi

**Gelecek Çalışmalar İçin Öneriler:**
1. Makine öğrenmesi modelleri ile yan etki tahminleme
2. Zaman serisi analizi ile tedavi süreçlerinin incelenmesi
3. İlişki kuralları madenciliği ile ilaç kombinasyonlarının analizi
4. Daha gelişmiş kümeleme algoritmaları (K-means, DBSCAN)
5. Boyut indirgeme teknikleri (PCA, t-SNE)

---
## Referanslar ve Kaynaklar

**Veri Seti:**
- Kaggle: 1000 Drugs and Side Effects Dataset
- URL: https://www.kaggle.com/datasets/palakjain9/1000-drugs-and-side-effects

**Kullanılan Kütüphaneler:**
- pandas: Veri manipülasyonu
- numpy: Sayısal hesaplamalar
- matplotlib: Görselleştirme
- seaborn: İleri görselleştirme
- scipy: İstatistiksel testler

**Metodolojiler:**
- Pearson ve Spearman korelasyon
- Hiyerarşik kümeleme (Ward metodu)
- Tanımlayıcı istatistikler
- Normalizasyon teknikleri

---
## Proje Bitişi

**Tarih:** 2024

**Yazar:** Veri Madenciliği Öğrencisi

**Teşekkürler:** Bu proje, veri madenciliği tekniklerinin gerçek dünya veri setlerine uygulanmasını göstermektedir.

---

---
## 📥 HTML Çıktısı Oluşturma

Aşağıdaki hücreyi çalıştırarak projenizin HTML çıktısını alabilirsiniz.

In [ ]:
# ============================================================================
# HTML ÇIKTI OLUŞTURMA VE İNDİRME
# ============================================================================
# Bu hücreyi çalıştırarak notebook'u HTML formatına çevirebilir ve
# Google Colab'da kullanıyorsanız otomatik olarak indirebilirsiniz.
# ============================================================================

print("🌐 HTML ÇIKTI OLUŞTURULUYOR...")
print("="*70)

# Google Colab'da mı çalışıyoruz?
try:
    import google.colab
    IN_COLAB = True
    print("✅ Google Colab ortamı tespit edildi")
except:
    IN_COLAB = False
    print("✅ Lokal Jupyter ortamı tespit edildi")

# nbconvert'i yükle
print("\n📦 Gerekli paketler kontrol ediliyor...")
import subprocess
import sys

try:
    import nbconvert
    print("   ✅ nbconvert mevcut")
except ImportError:
    print("   ⬇️ nbconvert yükleniyor...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "nbconvert", "-q"])
    print("   ✅ nbconvert yüklendi")

# HTML'e çevir
print("\n🔄 HTML dönüştürme başlıyor...")
import os
import glob

# Notebook dosyasını bul
notebook_files = glob.glob('*.ipynb')

if notebook_files:
    for nb_file in notebook_files:
        # .ipynb_checkpoints klasöründeki dosyaları atla
        if '.ipynb_checkpoints' not in nb_file:
            print(f"   📝 {nb_file} dönüştürülüyor...")
            
            # HTML'e çevir (kod hücreleri gizli)
            result = subprocess.run([
                'jupyter', 'nbconvert',
                '--to', 'html',
                nb_file,
                '--no-input',  # Kod hücrelerini gizle
                '--no-prompt'  # Hücre numaralarını gizle
            ], capture_output=True, text=True)
            
            if result.returncode == 0:
                html_file = nb_file.replace('.ipynb', '.html')
                if os.path.exists(html_file):
                    size_kb = os.path.getsize(html_file) / 1024
                    print(f"   ✅ {html_file} oluşturuldu ({size_kb:.1f} KB)")
                else:
                    print(f"   ⚠️ {html_file} oluşturulamadı")
            else:
                print(f"   ❌ Hata: {result.stderr}")
else:
    print("   ❌ Notebook dosyası bulunamadı!")

# HTML dosyalarını listele
html_files = glob.glob('*.html')
if html_files:
    print(f"\n📂 Oluşturulan HTML dosyaları ({len(html_files)} adet):")
    for html in html_files:
        size_kb = os.path.getsize(html) / 1024
        print(f"   ✅ {html} ({size_kb:.1f} KB)")
else:
    print("\n⚠️ HTML dosyası bulunamadı!")

# Google Colab'daysa indir
if IN_COLAB and html_files:
    print("\n📥 HTML dosyaları indiriliyor...")
    from google.colab import files
    
    for html_file in html_files:
        print(f"   ⬇️ {html_file}")
        files.download(html_file)
    
    print("\n" + "="*70)
    print("🎉 BAŞARILI! HTML dosyanız indirildi.")
    print("="*70)
    print("\n📂 HTML dosyanız tarayıcınızın İndirilenler klasöründe!")
    print("💡 Dosyaya çift tıklayarak açabilirsiniz.")
    
elif not IN_COLAB and html_files:
    print("\n" + "="*70)
    print("🎉 BAŞARILI! HTML dosyası oluşturuldu.")
    print("="*70)
    print(f"\n📂 Konum: {os.path.abspath(html_files[0])}")
    print("💡 Dosyaya çift tıklayarak açabilirsiniz.")
    
else:
    print("\n" + "="*70)
    print("❌ HTML oluşturulamadı!")
    print("="*70)
    print("\n🔧 Alternatif Yöntemler:")
    if IN_COLAB:
        print("\n   Manuel indirme:")
        print("   1. Sol menüden 📁 Dosyalar sekmesini açın")
        print("   2. .html dosyasını bulun")
        print("   3. Sağ tıklayıp 'İndir' seçin")
    else:
        print("\n   Jupyter menüsünden:")
        print("   File → Download as → HTML (.html)")

print("\n" + "="*70)